In [1]:
import numpy as np
import pandas as pd

from UpdateDB_ref import Endpoint_annotation as ep


*** CompoundDB module found. Will check the synonyms table to resolve CAS. ***



In [2]:
ep_refactored = ep.Endpoint(host='localhost', dbname='cii_test', user='postgres', password='PSGAdmin')

In [3]:
endpoints_ref = ep_refactored.get_endpoints()

In [4]:
ep_refactored.get_total_annotations_per_endpoint(endpoints_ref)

/home/emarch/miniconda3/envs/inditex/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Endpoints,YES,Pending,NO,No information
0,cmr,2684.0,66.0,567.0,4536.0
1,pbt,1881.0,0.0,213.0,5759.0
2,vpvb,57.0,0.0,378.0,7418.0
3,endocrine_disruptor,229.0,0.0,9.0,7615.0
4,sensitiser,65.0,72.0,0.0,4438.0
5,chem_id,0.0,0.0,0.0,0.0
6,c,1644.0,167.0,538.0,4257.0
7,m,1028.0,0.0,504.0,5074.0
8,r,1392.0,97.0,437.0,4680.0
9,p,3652.0,0.0,0.0,2954.0


In [5]:
endpoint_pos = pd.read_sql_query("""select pea.annotation_id, rn.names, pea.endpoint 
from positive_endpoint_annotations pea
left join regulation_names rn on rn.id = pea.annotation_id
order by pea.id """, ep_refactored.conn)

In [6]:
endpoint_dict = {'CMR':endpoint_pos.loc[endpoint_pos['endpoint'] == 'cmr','names'].tolist(),
                 'PBT':endpoint_pos.loc[endpoint_pos['endpoint'] == 'pbt','names'].tolist(),
                 'vPvB':endpoint_pos.loc[endpoint_pos['endpoint'] == 'vpvb','names'].tolist(),
                 'Endocrine Disruptor':endpoint_pos.loc[endpoint_pos['endpoint'] == 'endocrine disruptor','names'].tolist()}

In [7]:
chem_ids = ep_refactored.get_chem_id_dataframe()

In [8]:
subs = ep_refactored.get_curated_substances()

In [9]:
len(chem_ids)

8811

In [10]:
len(subs)

7853

In [11]:
len(subs[['class_name_curated','preferred_name_curated','mol_formula_curated']].drop_duplicates())

7834

In [12]:
len(chem_ids.loc[chem_ids['chem_type_id'] == 1, :])

7852

In [13]:
subs.loc[subs['preferred_name_curated'] == "Silicon dioxide"].sort_values(by='id')

,id,chem_id,class_name_curated,preferred_name_curated,mol_formula_curated
672,4639,1781,Textile Chemicals,Silicon dioxide,O2Si


In [14]:
rep_subs = subs.loc[(subs['id'] > 4634) &
                    (~subs['preferred_name_curated'].isna()), 'preferred_name_curated'].unique()

In [15]:
repeated_subs = subs.loc[subs['preferred_name_curated'].isin(rep_subs)].sort_values(by='preferred_name_curated')

In [16]:
# repeated_subs.to_excel('repeated_subs_cii.xlsx')

In [17]:
chem_ids_unique = chem_ids.loc[chem_ids['chem_type_id'] == 1, 'id'].unique()

In [18]:
%%time
chemid_endpoint = ep_refactored.get_annotations_for_chem_id(chem_ids_unique, endpoint_dict)

CPU times: user 2min 55s, sys: 1.86 s, total: 2min 57s
Wall time: 10min 13s


In [19]:
len(chemid_endpoint)

7852

In [20]:
chemid_endpoint.sort_values(by='chem_id').head()

,chem_id,CMR,PBT,vPvB,Endocrine Disruptor
24,1.0,YES,NO,NO,No information
25,2.0,No information,No information,No information,No information
26,5.0,YES,NO,NO,YES
27,7.0,YES,NO,NO,No information
28,9.0,YES,NO,NO,No information


In [21]:
len(endpoints_ref)

7853

In [22]:
endpoints_ref.sort_values(by='chem_id').head()

,id,cmr,pbt,vpvb,endocrine_disruptor,sensitiser,chem_id,c,m,r,p,b,t,vp,vb,androgen_rc,estrogen_rc
0,0,YES,NO,NO,No information,YES,1,NO,YES,No information,YES,YES,YES,No information,No information,nan,nan
4633,4652,No information,No information,NO,No information,YES,2,No information,No information,No information,No information,No information,No information,No information,No information,nan,nan
1,1,YES,NO,NO,YES,No information,5,NO,YES,YES,YES,No information,No information,No information,No information,nan,nan
2,2,YES,YES,NO,No information,No information,7,YES,YES,No information,YES,YES,YES,No information,No information,nan,nan
3,3,YES,No information,No information,No information,No information,9,YES,No information,No information,No information,No information,No information,No information,No information,nan,nan


In [23]:
len(chemid_endpoint)

7852

### Adding annotations to DB
After getting the endpoint annotations per substance, and at the same time we update the database with them, the check function should come.
Before updating the DB, the check function should see if there's a substance id in the table.
If no, adds the new substance with the annotations.
If there is, then it checks the annotations and applies the filters we've spoken

In [24]:
# ep.add_annotations_to_database(subs_endpoint)

### Total annotation check
This only counts the total number of annotations per endpoint we have

In [25]:
ep_refactored.get_total_annotations_per_endpoint(chemid_endpoint)

,Endpoints,YES,Pending,NO,No information
0,CMR,2271.0,71.0,0.0,5510.0
1,PBT,10.0,0.0,1258.0,6584.0
2,vPvB,18.0,0.0,1243.0,6591.0
3,Endocrine Disruptor,144.0,0.0,0.0,7708.0
